# 📄✨ ResuMatch: A RAG-Powered Resume Tailoring Assistant
> *An agentic GenAI system that analyzes, matches, and rewrites resumes for any job — intelligently and interactively.*





### ✅ Phase 1: MVP with Single Job Description (Recommended to Start)
📄 Upload Resume (PDF)

🔗 Paste Job Description URL or text

🧠 Extract and analyze JD

📊 Compute match score

✍️ Tailor resume 

💬 Chat-based refinement (user feedback loop)






📌 This notebook defines a complete Retrieval-Augmented Generation (RAG) pipeline for intelligent resume matching. It uses LangChain agents, Gemini (Google GenAI) to:
- Extract and analyze resumes
- Compare with job descriptions
- Identify skill gaps
- Interact with users to fill in missing skills
- Generate a tailored resume 




### 🔁 Future Scope: Scrape & Rank Top 10 Jobs (LinkedIn, etc.)

* Automatically scrape multiple jobs (via scraping or API)

* Rank them by match score

* Present a ranked list with links, highlights, and an automatic “Apply” buttons.

* The entire agentic workflow should be automated and will be deployed with an U/I (eg. gradio)

* Generate a cover letter also to highlight the credential with a personal touch. 



###  📊 Visual Workflow Overview 


1. Resume Upload

   
   └── Extract text from PDF using PyMuPDF via LangChain agent

   

2. Job Description Input

   
   └── Paste or upload JD, cleaned and processed

   

3. Semantic Match Scoring

   
   └── Gemini prompt-based scoring (0–100) via LangChain tool

   

4. Skill Gap Analysis

   
   └── Gemini identifies 8–10 missing skills not present in the resume

5. Interactive Feedback Agent

    
   └── User engages with each missing skill via chatbot interface
       → Describes experience or opts to skip

   

6. Resume Rewriting Agent

    
   └── Gemini re-generates the resume using:
       - Original resume text
       - Job description
       - User feedback (RAG prompt)
       

7. Output: Tailored Resume
    
   └── Save as:
       - .docx (with formatting, bullets, bold headers)
       - .tex (moderncv LaTeX format)


# Gen AI Intensive Course Capstone 2025Q1


### NoteBook by Debisree Ray

April 2025

https://www.kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1

This notebook is the capstone project work

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-resume/DR.pdf


In [2]:
#!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.7 MB/s eta 0:00:00


In [3]:
!pip install -q --upgrade google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.2 MB/s eta 0:00:00


In [4]:
%pip install PyMuPDF google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 74.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.5 MB/s eta 0:00:00


In [6]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=9357faff8f5a4bd644fe25849a059b8de035c11ad0968ba53c675e5c332faf0c
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [7]:
%pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import google
from google import genai
from google.genai import types
import fitz  # PyMuPDF

import os

#from selenium import webdriver
#from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.common.by import By
import time

from IPython.display import Markdown, display
from fpdf import FPDF
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

from langchain_core.tools import tool
from sentence_transformers import SentenceTransformer, util

import re
import gradio as gr

#import google.generativeai as genai
import requests



from pprint import pprint  # optional, for prettier output
from IPython.display import HTML, Markdown, display

genai.__version__

2025-04-21 05:37:20.295341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745213840.572146      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745213840.644210      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'1.7.0'

In [9]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

### Secret API Key

In [10]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [11]:
client = genai.Client(api_key=GOOGLE_API_KEY)


# ✅ Step 1: Resume Parser (PDF to Text)

The goal is to take the resume in pdf and convert that in a more usable format. 

In [12]:
# === 📄 Resume Extractor ===
@tool
def resume_extraction_agent(file_path: str) -> str:
    """Reads a PDF resume from the given path and returns the extracted text."""
    with open(file_path, 'rb') as f:
        doc = fitz.open(stream=f.read(), filetype="pdf")
        text = "".join([page.get_text() for page in doc])
    return text



resume_text = resume_extraction_agent.invoke({'file_path': '/kaggle/input/my-resume/DR.pdf'} )


### A summary generator 

We don't actually need it, directly for the project purpose - however, it can be useful.

It can be used for any quick reference

In [13]:
# === 📄 Resume summarizer ===

def summarize_resume_with_gemini(resume_text):
    prompt = f"""
    You are an expert resume summarizer. Extract the following from the resume:
    - Full name
    _ Highest Degree
    - Email (if available)
    - Phone (if available)
    - Top 10 skills
    _ Tech Stack
    - Preferred job titles
    - Years of experience
    - Last three job roles with company and duration
    - Education summary

    Resume:
    {resume_text}
    """
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt)

 
    return response.text

summary = summarize_resume_with_gemini(resume_text)
print(summary)

Here's a summary of the resume, extracting the information you requested:

*   **Full Name:** Debisree Ray
*   **Highest Degree:** Ph.D.
*   **Email:** debisreer@gmail.com
*   **Phone:** 662-694-1319
*   **Top 10 Skills:**
    1.  Data Wrangling
    2.  Exploratory Data Analysis (EDA)
    3.  Data Visualization
    4.  Machine Learning (Regression/Classification)
    5.  Deep Learning
    6.  Web Application Development (Python-Flask)
    7.  MLOps (CI/CD Pipelines)
    8.  Statistical Data Analytics
    9.  Scientific Computing
    10. Technical Writing
*   **Tech Stack:** Python, Flask, SQL, Azure, Databricks, Anaconda/Jupyter, Pandas, NumPy, Seaborn, SciPy, Matplotlib, Plotly, MLflow, Azure DevOps, Unity-3D. Fortran, R, LaTeX, Mathematica, MATLAB, XMRACE, GnuPlot
*   **Preferred Job Titles:** Data Scientist
*   **Years of Experience:** 6+ years (Post Ph.D. experience based on dates provided, not including teaching/research assistant roles)
*   **Last Three Job Roles:**
    *   Data 

# ✅ Step 2: Job Description from Linkedin

The goal is to take the JD convert that in a more usable format. 

Linkedin does not provide publicly available API. To begin with we are manually copying and pasting one particular job description that the user is interested in. 

We can take a screen-shot as well and retreive from the image. However, this has not done here. 


In [14]:
# === 🧾 Job Description Loader ===

@tool
def jd_loader_agent(jd_text) -> str:
    """Cleans and returns job description text from raw pasted input."""
    return jd_text.strip()

jd_text = jd_loader_agent.invoke({"jd_text":"""" About the job Position Summary

What you'll do

We are looking for a Senior Data Scientist to join our science team and develop state-of-the-art models to power Product Search and Recommendation across Walmart’s multiple international markets. Our mission is to provide our customers with the most delightful user experience throughout their online shopping journey with us. We create powerful AI and ML powered solutions for highly intricate real-world problems. You will be working on these industry-defining problems and have the opportunity to push the frontiers of Search & Recommendation Science by advancing areas including AI, ML, NLP, Optimization, Algorithm and Customer-facing large system and service designs.

What You Will Do

Bring in bleeding-edge academic research and industry practices and demonstrate originality and creativity in developing novel solutions for Search or Recommendation Systems.
Partner with other data scientists, engineers and product managers to design, train, test and deploy machine learning models.
Work on projects with substantial ambiguity and translate business and engineering requirements to identify and define a clear roadmap to deliver the ML models.
Perform rigorous offline and A/B testing of your models and communicate your findings to technical and non-technical stakeholders, both verbally and through written communications.
You will help maintain models, improve them as needed to bring in cutting-edge research, within a highly scalable infrastructure.
You will also help new data scientist’s onboarding and mentorship.
You are strongly encouraged to publish your research and novel findings in internal and external research and industry conferences and journals, and file patent applications.

What You Will Bring

Have a MS/PhD in relevant technical degree (preferably Computer Science, Machine Learning, Operations Research, Applied Mathematics, Statistics, Engineering etc.) with 3+ years of relevant work experience.
Must have a deep background in NLP, Search Science, Recommendation Systems, Machine Learning, Deep Learning, Optimization, Algorithm and Software development.
It is a bonus to have prior knowledge of Search/Recommendation Systems related technology and Generative AI.
Sound knowledge of Python, SQL, PySpark, Machine Learning Libraries, Big Data processing systems in cloud environment is expected.
A track record of building ML models or delivering impactful customer-facing products.
Experience of publications in peer-reviewed conferences and journals or patent filings.

At Walmart, we offer competitive pay as well as performance-based bonus awards and other great benefits for a happier mind, body, and wallet. Health benefits include medical, vision and dental coverage. Financial benefits include 401(k), stock purchase and company-paid life insurance. Paid time off benefits include PTO (including sick leave), parental leave, family care leave, bereavement, jury duty, and voting. Other benefits include short-term and long-term disability, company discounts, Military Leave Pay, adoption and surrogacy expense reimbursement, and more.



You will also receive PTO and/or PPTO that can be used for vacation, sick leave, holidays, or other purposes. The amount you receive depends on your job classification and length of employment. It will meet or exceed the requirements of paid sick leave laws, where applicable.



For information about PTO, see https://one.walmart.com/notices.


Live Better U is a Walmart-paid education benefit program for full-time and part-time associates in Walmart and Sam's Club facilities. Programs range from high school completion to bachelor's degrees, including English Language Learning and short-form certificates. Tuition, books, and fees are completely paid for by Walmart.



Eligibility requirements apply to some benefits and may depend on your job classification and length of employment. Benefits are subject to change and may be subject to a specific plan or program terms.



For Information About Benefits And Eligibility, See One.Walmart.



The annual salary range for this position is $117,000.00-$234,000.00


Additional Compensation Includes Annual Or Quarterly Performance Bonuses.


Additional Compensation For Certain Positions May Also Include


 Stock

Minimum Qualifications...

Outlined below are the required minimum qualifications for this position. If none are listed, there are no minimum qualifications. 

Option 1- Bachelor’s degree in Statistics, Economics, Analytics, Mathematics, Computer Science, Information Technology, or related field and 3 years' experience in an analytics related field. Option 2- Master’s degree in Statistics, Economics, Analytics, Mathematics, Computer Science, Information Technology, or related field and 1 years' experience in an analytics related field. Option 3 - 5 years' experience in an analytics or related field.

Preferred Qualifications...

Outlined below are the optional preferred qualifications for this position. If none are listed, there are no preferred qualifications.

Data science, machine learning, optimization models, Master’s degree in Machine Learning, Computer Science, Information Technology, Operations Research, Statistics, Applied Mathematics, Econometrics, Successful completion of one or more assessments in Python, Spark, Scala, or R, Using open source frameworks (for example, scikit learn, tensorflow, torch), We value candidates with a background in creating inclusive digital experiences, demonstrating knowledge in implementing Web Content Accessibility Guidelines (WCAG) 2.2 AA standards, assistive technologies, and integrating digital accessibility seamlessly. The ideal candidate would have knowledge of accessibility best practices and join us as we continue to create accessible products and services following Walmart’s accessibility standards and guidelines for supporting an inclusive culture.

Primary Location...

680 West California Avenue, Sunnyvale, CA 94086-4834, United States of America """})




# ✅ Step 3: Comparing the resume text and JD to generate a similarity score

### The goal is to analyze how well the resume is aligned with the JD. 



LangChain Agent: Match Score Evaluator

**What is a LangChain Agent?**
  
A LangChain agent is a smart reasoning system that can decide which tools or prompts
to use dynamically based on the input. It acts like a smart assistant that chains multiple
reasoning steps together (e.g., parse > search > score > refine). In this case, we wrap the matching score process into an agent that:
- Accepts the resume and job description
- Calls Gemini to produce a numeric match score
- Optionally returns reasoning/explanation as a separate step


This makes our RAG engine modular and extendable to tool-based pipelines (e.g., memory, retrieval, editing).


In [15]:

@tool
def match_score_agent(resume_text: str, jd_text: str) -> float:
    """Uses Gemini to return a 0–100 numeric score indicating how well the resume matches the job description."""
    prompt = f"""
You are an expert AI evaluator. Score how well the following resume matches the job description on a scale of 0 to 100.
Return only the numeric score. Do not include explanation.

Resume:
{resume_text}

Job Description:
{jd_text}
"""
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt)
    try:
        return round(float(response.text.strip()), 2)
    except ValueError:
        print("⚠️ Unable to parse score from model output:", response.text)
        return None


score= match_score_agent.invoke({'resume_text': 'resume_text', 'jd_text': 'jd_text'} )
score

78.0

# ✅ Step 4: Skill Gap Analyzer - LangChain Agentic Skill Analyzer

I want to give the user point-wise input, the gap in the skills between the resume and the JD


In [16]:
@tool
def missing_skills_agent(resume_text: str, jd_text: str) -> str:
    """Uses Gemini to identify missing skills from the job description compared to the resume."""
    prompt = f"""
You are an AI assistant that compares job descriptions to resumes.

Resume:
{resume_text}

Job Description:
{jd_text}

Identify the most important 8–10 skills or qualifications from the job description that are clearly missing from the resume.

For each point:
1. Start with a numbered index (1., 2., etc.)
2. Clearly name the missing skill
3. Provide a short but specific explanation
4. Format the entire output exactly as:

1. Skill Name: Explanation

Each point MUST begin with the number and a dot, followed by the skill name, then a colon, and the explanation in one paragraph. Be consistent across runs.
"""
   
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt)
    return response.text.strip()
   

# Run skill matching and render the markdown


missing_skills = missing_skills_agent.invoke({"resume_text": resume_text, "jd_text": jd_text})
display(Markdown(f"### ❌ Missing Skills or Keywords\n\n{missing_skills}"))

### ❌ Missing Skills or Keywords

Here's a comparison of the resume and job description, highlighting the missing skills:

1.  NLP: The resume mentions general data science and machine learning but lacks explicit mention of Natural Language Processing experience, a core requirement for the position.
2.  Search Science: While the resume mentions experience building ML models, it does not demonstrate specific experience in search science, which is a primary focus of the role.
3.  Recommendation Systems: Similar to search science, the resume lacks specific experience in building or deploying recommendation systems.
4.  Generative AI: The job description mentions prior knowledge of Generative AI as a bonus, and this is not mentioned in the resume.
5.  PySpark: Although the resume mentions Python, SQL, and general data science tools, it does not explicitly mention PySpark, which is listed as an expected skill.
6.  Experience with customer-facing products: The job description requires a track record of building ML models or delivering impactful customer-facing products, which the resume does not highlight adequately. The resume mainly mentions internal dashboards.
7.  Experience with large systems: The job description mentions working in "customer-facing large system and service designs," while the resume lacks explicit experience or contributions to large-scale systems.
8.  Accessibility knowledge (WCAG 2.2 AA): The job description mentions preferred qualifications in creating inclusive digital experiences and knowledge of Web Content Accessibility Guidelines (WCAG) 2.2 AA standards, which are not mentioned in the resume.

# ✅ Step 5: User Feedback Agent- A conversational bot to collect user's input regarding all missing points

As the workflow pointed out all skill gaps (between the resume & the JD), this is a conversation between with the agent and the user to get more clarity about the 'gaps'. 

In [17]:
@tool
def user_feedback_agent(missing_skills: str) -> list:
    """Conversational Gemini agent that collects user feedback naturally for each missing skill."""
    print("👋 Hello! Let's work together to refine your resume based on the missing skills I found.")
    print("💬 For each skill, you can respond freely — no need for yes/no. Just tell me how you'd like to represent your experience.")
    print("✅ Type 'pass' to skip or 'end' to finish anytime.")

    feedback_list = []
    skills = re.findall(r"\d+\.\s+(.*?):\s+(.*?)(?=\n\d+\.\s|\Z)", missing_skills, re.DOTALL)

    if not skills:
        print("⚠️ No skills parsed from the input. Please check formatting.")
        return feedback_list

    for i, (skill, explanation) in enumerate(skills, 1):
        print(f"\n🔹 {i}. {skill.strip()}\n📎 {explanation.strip()}")
        response = input("🗣️ How would you describe your experience for this skill? (or type 'pass'/'end'): ").strip()

        if response.lower() == 'end':
            print("🛑 Ending the session.")
            break
        elif response.lower() == 'pass' or not response:
            feedback_list.append((skill.strip(), None))
        else:
            feedback_list.append((skill.strip(), response))

    print("✅ Thanks! Your responses are saved.")
    return feedback_list

# Fake feedback for auto submission mode
def simulate_feedback(skills):
    return [(skill, "Simulated experience for submission.") for skill in skills]


is_kaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE") == "Batch"
if is_kaggle:
    # Extract skill names from parsed missing_skills_agent
    skills = ["no", "yes", 'yes', 'yes', 'yes', 'I have never worked with external clients. however, our stakeholders are interanl clients. i havebeen delivering Machine Learning based products.', "Most of my publications are in computational physics.", "No"]  # hardcoded or parsed
    feedback_list = simulate_feedback(skills)
else:
    feedback_list = user_feedback_agent.invoke({"missing_skills": missing_skills})



# ✅ Step 6: Tailored Resume Generator 

The final step is to generate a tailored and polished resume - starting from the original resume. The focus should be to align the resume towards the JD, where as the user feedback needs to be taken into account. 

In [18]:
@tool
def generate_final_resume_agent(resume_text: str, jd_text: str, user_feedback: list) -> float:
    """Uses Gemini with RAG-style prompt to generate a polished final resume and saves it as a DOCX file."""
    

    prompt = f"""
    You are an expert resume rewriting agent. Rewrite the resume to align with the job description.
    
    Return content in the following structure only:
    1. A centered bold heading with name: Debisree Ray, PhD
    2. A **single line** contact block: 📍 Memphis, TN | ✉️ debisreer@gmail.com | ☎️ 662-694-1319 | 🔗 LinkedIn: https://www.linkedin.com/in/debisree-ray-ph-d-82241355/ | 🧠 Kaggle: https://www.kaggle.com/debisree | 💻 GitHub: https://github.com/debisree
    3. Section: SUMMARY — short overview of profile
    4. Section: SKILLS — key skills
    5. Section: WORK EXPERIENCE — include job title, company, and bullet points
    6. Section: EDUCATION — list degrees
    7. Section: PROJECTS — if applicable
    8. Section: LEADERSHIP & VOLUNTEERING — include relevant experience
    
    Use bold headings, bullet points, and concise language. Do NOT include labels like 'Resume:', 'Job Description:', or explanations. Just return the resume content below.
    
    Resume:
    {resume_text}

    Job Description:
    {jd_text}
    
    Skills/Feedback to integrate:
    {feedback_list}
    """

 
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt)
    final_resume = response.text.strip()

    doc = Document()
    section_titles = ["SUMMARY", "SKILLS", "WORK EXPERIENCE", "EDUCATION", "PROJECTS", "LEADERSHIP & VOLUNTEERING"]

    for i, line in enumerate(final_resume.splitlines()):
        line = line.strip()
        if i == 0:
            para = doc.add_paragraph()
            para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            run = para.add_run(line)
            run.bold = True
            run.font.size = Pt(16)
        elif line.startswith("📍") or line.startswith("✉️") or "linkedin.com" in line:
            para = doc.add_paragraph()
            para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            run = para.add_run(line)
            run.font.size = Pt(10.5)
        elif line.upper() in section_titles:
            para = doc.add_paragraph()
            run = para.add_run(line.upper())
            run.bold = True
            run.font.size = Pt(12)
            doc.add_paragraph().add_run("―" * 80)
        elif line.startswith("- "):
            para = doc.add_paragraph(style='List Bullet')
            run = para.add_run(line[2:])
            run.font.size = Pt(11)
        elif line == "":
            doc.add_paragraph()
        else:
            para = doc.add_paragraph()
            run = para.add_run(line)
            run.font.size = Pt(11)

    #doc.save("AI_Gen_Resume.docx")
    doc.save("/kaggle/working/AI_Gen_Resume.docx")

    print("✅ Resume saved as: AI_Gen_Resume.docx")
    return final_resume

resume=generate_final_resume_agent.invoke({"resume_text": resume_text, "jd_text": jd_text, "user_feedback": feedback_list})


✅ Resume saved as: AI_Gen_Resume.docx


Docx does not look very nicely formatted - so trying the tex file generation. 

In [19]:
@tool
def generate_final_resume_tex_agent(resume_text: str, jd_text: str, user_feedback: list) -> str:
    """Uses Gemini and moderncv LaTeX to generate and save a styled .tex resume."""

    # Prepare feedback
    feedback_text = "\n".join([f"{skill}: {detail}" for skill, detail in user_feedback if detail])

    # Gemini prompt
    prompt = f"""
You are a resume rewriting assistant. Given a resume, a job description, and user feedback about missing skills,
generate a resume body that is well-structured and compact — optimized for LaTeX formatting.

Please use the following order:
- Summary
- Skills
- Work Experience
- Education
- Projects (optional)
- Leadership & Volunteering

Just return clean plain text lines — section titles, bullet points, no markdown.

Resume:
{resume_text}

Job Description:
{jd_text}

User Feedback:
{feedback_text}
"""

    
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt)
    resume_body = response.text.strip()

    # Assemble moderncv template
    tex_template = f"""
\\documentclass[11pt,a4paper,sans]{{moderncv}}
\\moderncvstyle{{classic}}
\\moderncvcolor{{blue}}

\\usepackage[utf8]{{inputenc}}
\\usepackage[scale=0.75]{{geometry}}
\\name{{Debisree}}{{Ray, PhD}}
\\email{{debisreer@gmail.com}}
\\phone{{662-694-1319}}
\\social[linkedin]{{debisree-ray-ph-d-82241355}}
\\social[kaggle]{{debisree}}
\\social[github]{{debisree}}

\\begin{{document}}

\\makecvtitle

\\section{{Resume}}
{resume_body}

\\end{{document}}
"""

    tex_path = "/kaggle/working/AI_Gen_Resume.tex"
    with open(tex_path, "w", encoding="utf-8") as f:
        f.write(tex_template)

    print(f"✅ Saved LaTeX resume to: {tex_path}")
    return resume_body


resume=generate_final_resume_tex_agent.invoke({"resume_text": resume_text, "jd_text": jd_text, "user_feedback": feedback_list})

✅ Saved LaTeX resume to: /kaggle/working/AI_Gen_Resume.tex


In [20]:
def sanitize_latex(text):
    replacements = {
        "&": r"\&",
        "%": r"\%",
        "$": r"\$",
        "#": r"\#",
        "_": r"\_",
        "{": r"\{",
        "}": r"\}",
        "~": r"\textasciitilde{}",
        "^": r"\textasciicircum{}",
        "\\": r"\textbackslash{}"
    }
    for char, replacement in replacements.items():
        text = text.replace(char, replacement)
    return text

@tool
def generate_final_resume_tex_agent(resume_text: str, jd_text: str, user_feedback: list) -> str:
    """Uses Gemini and moderncv LaTeX to generate and save a styled .tex resume with LaTeX-safe formatting."""

    # Prepare feedback
    feedback_text = "\n".join([f"{skill}: {detail}" for skill, detail in user_feedback if detail])

    # Gemini prompt
    prompt = f"""
    You are a resume rewriting assistant. Given a resume, a job description, and user feedback about missing skills,
    generate a resume body that is well-structured and compact — optimized for LaTeX formatting.
    
    Please use the following order:
    - Summary
    - Skills
    - Work Experience
    - Education
    - Projects (optional)
    - Leadership and Volunteering
    
    Use plain text lines — section titles, bullet points, no markdown or symbols.
    
    Resume:
    {resume_text}
    
    Job Description:
    {jd_text}
    
    User Feedback:
    {feedback_text}
    """

   
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt)
    resume_body = response.text.strip()
    resume_body = sanitize_latex(resume_body)

    # Build LaTeX using moderncv
    tex_template = f"""
\\documentclass[11pt,a4paper,sans]{{moderncv}}
\\moderncvstyle{{classic}}
\\moderncvcolor{{blue}}

\\usepackage[utf8]{{inputenc}}
\\usepackage[scale=0.75]{{geometry}}
\\name{{Debisree}}{{Ray, PhD}}
\\email{{debisreer@gmail.com}}
\\phone{{662-694-1319}}
\\social[linkedin]{{debisree-ray-ph-d-82241355}}
\\social[kaggle]{{debisree}}
\\social[github]{{debisree}}

\\begin{{document}}
\\makecvtitle

\\section{{Resume}}
{resume_body}

\\end{{document}}
"""

    tex_path = "/kaggle/working/AI_Gen_Resume.tex"
    with open(tex_path, "w", encoding="utf-8") as f:
        f.write(tex_template)

    print(f"✅ Saved LaTeX resume to: {tex_path}")
    return resume_body


resume=generate_final_resume_tex_agent.invoke({"resume_text": resume_text, "jd_text": jd_text, "user_feedback": feedback_list})

✅ Saved LaTeX resume to: /kaggle/working/AI_Gen_Resume.tex


### The tex file needs to run and needs a series of formatting improvement to look like a real resume! 

### 🙏 Acknowledgments
Special thanks to Kaggle and Google for providing the compute resources and access to cutting-edge tools like Gemini. This capstone project wouldn’t have been possible without the platform support and generous APIs.